# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





In [1]:
import datetime

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup

### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [2]:
browser = webdriver.Chrome(executable_path='./chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()

In [4]:
html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

so2_data = {}
item_type = ''

table_tr = table.find_all('tr')
table_td = [tr.find_all('td') for tr in table_tr]

for td in table_td:
    if len(td) == 5:
        item_type = td[0].text
    if item_type == 'SO2' and len(td) > 0:
        td_date = datetime.datetime.strptime(td[-3].text, '%Y/%m')
        if td_date.month < 9:
            so2_data.update({td_date.strftime('%Y/%m'): td[-2].text})

so2_data

{'2018/01': '1.80',
 '2018/02': '1.90',
 '2018/03': '2.20',
 '2018/04': '2.30',
 '2018/05': '3.10',
 '2018/06': '2.70',
 '2018/07': '2.20',
 '2018/08': '2.40'}

### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [5]:
so2_data = {}
co_data = {}

for td in table_td:
    if len(td) == 5:
        item_type = td[0].text
    if item_type == 'SO2' and len(td) > 0:
        td_date = datetime.datetime.strptime(td[-3].text, '%Y/%m')
        if td_date.month < 9:
            so2_data.update({td_date.strftime('%Y/%m'): td[-2].text})
    if item_type == 'CO' and len(td) > 0:
        td_date = datetime.datetime.strptime(td[-3].text, '%Y/%m')
        if td_date.month < 9:
            co_data.update({td_date.strftime('%Y/%m'): td[-2].text})

print('SO2 data:', so2_data)
print('CO data:', co_data)

SO2 data: {'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40'}
CO data: {'2018/01': '0.34', '2018/02': '0.44', '2018/03': '0.40', '2018/04': '0.38', '2018/05': '0.34', '2018/06': '0.29', '2018/07': '0.21', '2018/08': '0.30'}
